In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class MyNet(nn.Module):
  def __init__(self):
    super(MyNet,self).__init__()
    self.block1 = nn.Sequential(
        nn.Conv2d(1,12,3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(12),
        nn.Dropout2d(0.1),
        nn.Conv2d(12,16,3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.MaxPool2d(2,2),
        nn.Dropout2d(0.1)
     )

    self.block2 = nn.Sequential(
        nn.Conv2d(16,20,3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(20),
        nn.Dropout2d(0.1),
        nn.Conv2d(20,28,3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(28),
        nn.MaxPool2d(2,2),
        nn.Dropout2d(0.1)

    )

    self.block3 = nn.Sequential(
        nn.Conv2d(28,24,3),
        nn.ReLU(),
        nn.BatchNorm2d(24),
        nn.Dropout2d(0.1),
        nn.Conv2d(24,12,3),
        nn.ReLU(),
        nn.BatchNorm2d(12)
    )

    self.fc = nn.Linear(108,10)

  def forward(self, x):
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)

    x = x.view(x.size(0), -1)
    x = self.fc(x)
    x = F.log_softmax(x, dim=1)
    return x




















In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(16, 20, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(20, 24, 3, padding=1)
        self.conv4 = nn.Conv2d(24, 28, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(28, 14, 3)
        self.conv6 = nn.Conv2d(14, 12, 3)
        self.conv7 = nn.Conv2d(12, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = MyNet().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
              ReLU-2           [-1, 12, 28, 28]               0
       BatchNorm2d-3           [-1, 12, 28, 28]              24
         Dropout2d-4           [-1, 12, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,744
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
         MaxPool2d-8           [-1, 16, 14, 14]               0
         Dropout2d-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 20, 14, 14]           2,900
             ReLU-11           [-1, 20, 14, 14]               0
      BatchNorm2d-12           [-1, 20, 14, 14]              40
        Dropout2d-13           [-1, 20, 14, 14]               0
           Conv2d-14           [-1, 28,

In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 76901627.24it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 122359286.69it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 22756904.00it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7227059.47it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:

model = MyNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.12389836460351944 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.95it/s] 



Test set: Average loss: 0.0495, Accuracy: 9839/10000 (98.39%)



loss=0.030199863016605377 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.63it/s]



Test set: Average loss: 0.0350, Accuracy: 9879/10000 (98.79%)



loss=0.03657020628452301 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.74it/s] 


...


Test set: Average loss: 0.0186, Accuracy: 9936/10000 (99.36%)



loss=0.02186146192252636 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.58it/s]  



Test set: Average loss: 0.0191, Accuracy: 9936/10000 (99.36%)



loss=0.05158565938472748 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.51it/s]  



Test set: Average loss: 0.0184, Accuracy: 9932/10000 (99.32%)

